## Dependencias

In [2]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score


## Espacio de decisión

In [3]:
neuronas_capa1_espacio = list(range(2, 200))
neuronas_capa2_espacio = list(range(2, 200))
funcion_activacion_espacio = ['relu', 'tanh', 'logistic']
tasa_aprendizaje_espacio = np.arange(0.0001, 0.1, 0.0001)

## Cargar datos 

In [4]:
datos = pd.read_sas('../data/churn.sas7bdat')
X = datos[[v for v in datos.columns if v[:2]=='V_']]
y = datos['TARGET']

## Escalar datos

In [5]:
escalador = StandardScaler()
X = escalador.fit_transform(X)

## Partición

In [6]:
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, 
                                                                        y, 
                                                                        test_size=0.3
                                                                        )

## Evaluación del modelo (función objetivo)

In [7]:
def evaluar_modelo(neuronas_capa1, neuronas_capa2, funcion_activacion, tasa_aprendizaje):
    modelo = MLPClassifier(hidden_layer_sizes=(neuronas_capa1, neuronas_capa2,),
                           activation=funcion_activacion,
                           learning_rate_init=tasa_aprendizaje,
                           max_iter=500)
    
    modelo.fit(X_entrenamiento, y_entrenamiento)
    y_score = modelo.predict_proba(X_prueba)[:,1]
    auc = roc_auc_score(y_prueba, y_score)
    return auc

## Inicializar población 

In [8]:
def inicializar_poblacion(tamano_poblacion):
    poblacion = []
    for _ in range(tamano_poblacion):
        neuronas_capa1 = random.choice(neuronas_capa1_espacio)
        neuronas_capa2 = random.choice(neuronas_capa2_espacio)
        funcion_activacion = random.choice(funcion_activacion_espacio)
        tasa_aprendizaje = random.choice(tasa_aprendizaje_espacio)
        poblacion.append([neuronas_capa1, neuronas_capa2, funcion_activacion, tasa_aprendizaje])
    return poblacion

## Evaluar la población

In [9]:
def evaluar_poblacion(poblacion):
    puntajes_fitness = []
    for individuo in poblacion:
        neuronas_capa1, neuronas_capa2, funcion_activacion, tasa_aprendizaje = individuo
        fitness = evaluar_modelo(neuronas_capa1, neuronas_capa2, funcion_activacion, tasa_aprendizaje)
        puntajes_fitness.append(fitness)
    return puntajes_fitness

## Seleccionar Padres

In [10]:
def seleccionar_padres(poblacion, puntajes_fitness, num_padres):
    padres = [poblacion[idx] for idx in np.argsort(puntajes_fitness)[-num_padres:]]
    return padres

## Reproducción

In [11]:
def cruce(padres, tamano_descendencia):
    descendencia = []
    for _ in range(tamano_descendencia):
        padre1 = random.choice(padres)
        padre2 = random.choice(padres)
        
        punto_cruce = random.randint(1, len(padre1)-1)
        hijo = padre1[:punto_cruce] + padre2[punto_cruce:]
        descendencia.append(hijo)
    
    return descendencia

## Mutación

In [12]:
def mutacion(descendencia, tasa_mutacion=0.1):
    for individuo in descendencia:
        if random.random() < tasa_mutacion:
            individuo[0] = random.choice(neuronas_capa1_espacio)
        if random.random() < tasa_mutacion:
            individuo[1] = random.choice(neuronas_capa2_espacio)
        if random.random() < tasa_mutacion:
            individuo[2] = random.choice(funcion_activacion_espacio)
        if random.random() < tasa_mutacion:
            individuo[3] = random.choice(tasa_aprendizaje_espacio)
    return descendencia


## Algoritmo Genético

In [13]:
def algoritmo_genetico(tamano_poblacion, num_generaciones, num_padres):
    poblacion = inicializar_poblacion(tamano_poblacion)
    
    for generacion in range(num_generaciones):
        print(f"Generación {generacion + 1}")
        
        puntajes_fitness = evaluar_poblacion(poblacion)
        print(f"Mejor AUC en esta generación: {max(puntajes_fitness)}")
        
        padres = seleccionar_padres(poblacion, puntajes_fitness, num_padres)
        
        tamano_descendencia = tamano_poblacion - num_padres
        descendencia = cruce(padres, tamano_descendencia)
        
        descendencia = mutacion(descendencia)
        
        poblacion = padres + descendencia
    
    puntajes_fitness_finales = evaluar_poblacion(poblacion)
    mejor_solucion_idx = np.argmax(puntajes_fitness_finales)
    mejor_solucion = poblacion[mejor_solucion_idx]
    
    print("Mejores hiperparámetros encontrados:")
    print(f"Neuronas en capa 1: {mejor_solucion[0]}")
    print(f"Neuronas en capa 2: {mejor_solucion[1]}")
    print(f"Función de activación: {mejor_solucion[2]}")
    print(f"Tasa de aprendizaje: {mejor_solucion[3]}")
    print(f"AUC: {puntajes_fitness_finales[mejor_solucion_idx]}")


In [14]:
tamano_poblacion = 20
num_generaciones = 5
num_padres = 4

algoritmo_genetico(tamano_poblacion, num_generaciones, num_padres)


Generación 1


/Users/josegustavofuentescabrera/entornos/si/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Mejor AUC en esta generación: 0.8160337283887686
Generación 2
Mejor AUC en esta generación: 0.8199418036616967
Generación 3
Mejor AUC en esta generación: 0.8041183467142433
Generación 4
Mejor AUC en esta generación: 0.8298712883904679
Generación 5
Mejor AUC en esta generación: 0.8133787859479207
Mejores hiperparámetros encontrados:
Neuronas en capa 1: 100
Neuronas en capa 2: 87
Función de activación: tanh
Tasa de aprendizaje: 0.034
AUC: 0.8304872350367444
